# 17/05 - Pair Programming ETL 2 - Guadalupe & Lara

In [7]:
import pandas as pd
pd.options.display.max_columns = None

Tendréis que usar el csv attacks_limpieza_completa.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

- El csv con los ataques de tiburones que hemos estado limpiando hasta ahora. Usad el que no tenía las variables categóricas codificadas.
- El csv con los datos climáticos de los principales paises que tienen ataques de tiburones.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:
- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los paises que seleccionamos en la lección de ayer: USA, Australia, New Zealand, South Africa, Papua New Guinea
- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por pais.

Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. Como separar esta información en distintas columnas fue un problema al que nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. 
En caso de que en aquel ejercicio no lo consiguierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. 
⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [33]:
attacks_entero = pd.read_csv("attacks_limpieza_completa.csv", index_col= 0)

#Cogemos las primeras columnas para seguir el enunciado y no tener las variables categóricas codificadas:

attacks_cols = attacks_entero[['year', 'type', 'country', 'age', 'species_', 'fecha_limpia', 'fatal',
       'sex', 'latitud', 'longitud']]

attacks_cols.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882


In [34]:
attacks = attacks_cols[attacks_cols["country"].isin(["usa", "australia", "new zealand", "south africa", "papua nueva guinea"])]
attacks

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000
5,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000
...,...,...,...,...,...,...,...,...,...,...
6210,0,Unprovoked,south africa,27.0,Unespecific,Jul,Y,M,-28.816624,24.991639
6212,0,Unprovoked,usa,27.0,Unespecific,Unknown,N,F,39.783730,-100.445882
6213,0,Unprovoked,australia,27.0,Tiger shark,Unknown,Y,M,-24.776109,134.755000
6214,0,Unprovoked,australia,27.0,Tiger shark,Unknown,Y,M,-24.776109,134.755000


In [3]:
#Este debe de ser el resultado, creo, de lo que hagamos hoy. Así que no lo usamos. Y cargo el de ETL1.
df2 = pd.read_csv("datos_union_clima_ataques.csv", index_col=0)
df2.head(2)

,case_number,year,type,country,area,location,activity,name,age,time,...,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,...,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
1,2018.06.18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,11,14h00 -15h00,...,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875


In [72]:
clima = pd.read_csv("datos_clima.csv", index_col = 0)
clima.head(2)

,country,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,USA,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 245, 'speed':...",21,2,8,1017,none,0,0,200,2
1,USA,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 250, 'speed':...",21,2,8,1018,none,0,0,230,2


In [73]:
#Como en este el country aparece en mayúsculas, reciclamos una función para transformar a minúsculas:
def minusculizator (columna):
    try:
        columna = columna.lower()
    except:
        pass
    return columna

clima["country"] = clima["country"].apply(minusculizator)
clima.head(2)

,country,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,usa,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 245, 'speed':...",21,2,8,1017,none,0,0,200,2
1,usa,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 250, 'speed':...",21,2,8,1018,none,0,0,230,2


Separamos las columnas rh_profile y wind_profile, siguiendo el ejemplo del gitbook:

In [ ]:
''' import ast

clima['wind_profile'] = clima['wind_profile'].apply(ast.Str)
x = clima['wind_profile'].apply(pd.Series)
x'''


In [74]:
import ast

clima['wind_profile'] = clima['wind_profile'].apply(ast.literal_eval)

x = clima['wind_profile'].apply(pd.Series)


In [83]:
lista = []
for col in range(len(x)):
    lista.append(col)

In [85]:
a = pd.DataFrame(lista)

In [86]:
a

,0
0,0
1,1
2,2
3,3
4,4
...,...
315,315
316,316
317,317
318,318


In [76]:
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "wind_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["wind"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    clima.insert(i, nombre, valores)


KeyError: 'wind'

In [77]:
clima

,country,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,usa,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 245, 'speed':...",21,2,8,1017,none,0,0,200,2
1,usa,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 250, 'speed':...",21,2,8,1018,none,0,0,230,2
2,usa,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 250, 'speed':...",22,2,5,1018,none,0,0,140,2
3,usa,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 155, 'speed':...",21,2,5,1018,none,0,0,135,3
4,usa,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 7}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 155, 'speed':...",21,2,6,1016,none,0,0,135,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,papua_new_guinea,180,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 95, 'speed': ...",27,-4,10,1011,rain,2,0,90,4
60,papua_new_guinea,183,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 95, 'speed': ...",27,-4,10,1011,rain,2,0,85,4
61,papua_new_guinea,186,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,-4,10,1013,none,2,0,100,4
62,papua_new_guinea,189,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,-1,10,1011,none,2,0,100,4


In [ ]:

clima['rh_profile']= clima['rh_profile'].apply(ast.literal_eval)
y = clima['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)

# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 